In [7]:
import pymongo
import pandas as pd
from konlpy.tag import Komoran

client = pymongo.MongoClient("192.168.10.240", 27017)
db = client['AI_LKJ']
collection_APP_GOOGLE = db['TRAVEL_APPLICATION']
collection_APP_APPLE = db['TRAVEL_APPLICATION_APPLE']

data_APP_GOOGLE = list(collection_APP_GOOGLE.find())  # find()의 결과를 리스트로 변환
data_APP_APPLE = list(collection_APP_APPLE.find())  # find()의 결과를 리스트로 변환

df_APP_GOOGLE = pd.DataFrame(data_APP_GOOGLE)
df_APP_APPLE = pd.DataFrame(data_APP_APPLE)

In [8]:
df_app = pd.concat([df_APP_GOOGLE, df_APP_APPLE], ignore_index=True)

df_app['user_comments'] = df_app['user_comments'].replace('[^a-zA-Zㄱ-힣♥]', " ", regex=True).str.upper()

In [25]:
komoran = Komoran()  # Komoran 형태소 분석기 사용
sentiment_df = (df_app['user_comments'])
sentiment_df

0        낯선 여행지에서 맞든 틀리든 이런 가이드가 있다는 것은 많은 도움이 됩니다  때론 ...
1        웹사이트에서 구글로 로긴하고 예약한다음 메일에 티켓 다운로드하기가 있길래 폰에서 앱...
2        최근 업데이트후 실시간 항공권 가격이 낚시성 가격으로 나오고 결제는 실제로 되지 않...
3        여행전 항상 여행정보를 위해 열어보는 앱입니다  숙소및 식당은 물론이거니와 들러볼만...
4        다양한 콘텐츠와 뷰  리뷰가 잘 어울어진 어플  하지만  지극히 여행자 중심의 결과...
                               ...                        
17746    국내선 검색하니까 에어부산은 있길래 저가항공 다 나올줄알고 매번 검색해도 항상 에어...
17747                                            스카이팀도 핵좋음
17748    카카오톡으로 공유하기가 안됨 최신버전의 앱 아니라고만 나옴  스카이스캐너가 업데이트...
17749                        잘이용하고있습니다  신속하고 좋은 정보 항상 감사  
17750                                          완전 쵝오      
Name: user_comments, Length: 17751, dtype: object

In [24]:
def get_sentiment_score(comment):
    score = 0
    words = komoran.pos(comment)  # 형태소 분석
    for word, pos in words:
        if pos in ['VA', 'VV', 'VX', 'VCP', 'VCN', 'MAG']:  # 감성 단어인 경우
            found = sentiment_df[sentiment_df[0] == word]  # 단어 검색
            if not found.empty:  # 단어가 존재하는 경우
                polarity = found.iloc[0, 1]  # 극성(긍정/부정) 확인
                if polarity == 'P':
                    score += 1  # 긍정일 경우 감성 점수 +1
                elif polarity == 'N':
                    score -= 1  # 부정일 경우 감성 점수 -1
    return score

scores = df_app['user_comments'].apply(get_sentiment_score)
print(scores)  # 결과 출력

KeyError: False